In [ ]:
import numpy as np
import pandas as pd

import os
print(os.listdir("../input"))

import cv2
from tqdm import tqdm
import h5py

In [ ]:
path="../input/train/train"
new_path_train="../input/train/train_new_resized_96"
label=[]
data1=[]
counter=0

In [ ]:
#labeling the data
for file in os.listdir(path):
    image_data=cv2.imread(os.path.join(path,file), cv2.IMREAD_GRAYSCALE)
    image_data=cv2.resize(image_data,(96,96))
    if file.startswith("cat"):
        label.append(0) #labeling cats pictures with 0
    elif file.startswith("dog"):
        label.append(1) #labeling dogs pictures with 1
    try:
        data1.append(image_data/255)
    except:
        label=label[:len(label)-1]
    counter+=1
    if counter%1000==0:
        print (counter," image data labelled")

In [ ]:
from keras import Sequential
from keras.layers import Dense,MaxPooling2D,Conv2D,Flatten,Dropout

There are no silver bullets in terms of the CNN architecture design. The best way to find a model that’s appropriate for a specific case is to start with some basic design and iteratively improve it.

In [ ]:
model=Sequential()

model.add(Conv2D(kernel_size=(3,3),filters=3,input_shape=(96,96,1),activation="relu"))
model.add(Conv2D(kernel_size=(3,3),filters=10,activation="relu",padding="same"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(kernel_size=(3,3),filters=3,activation="relu"))
model.add(Conv2D(kernel_size=(5,5),filters=5,activation="relu"))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
model.add(Conv2D(kernel_size=(2,2),strides=(2,2),filters=10))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(100,activation="sigmoid"))
model.add(Dense(1,activation="sigmoid"))
model.summary()
model.compile(optimizer="adadelta",loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
import numpy as np

data1=np.array(data1)
print (data1.shape)
data1=data1.reshape((data1.shape)[0],(data1.shape)[1],(data1.shape)[2],1)
#data1=data1/255
labels=np.array(label)

In [ ]:
print (data1.shape)
print (labels.shape)

In [ ]:
model.fit(data1,labels,validation_split=0.2,epochs=20,batch_size=10)
model.save_weights("model.h5")

In [ ]:
test_data=[]
id=[]
counter=0
for file in os.listdir("../input/test1/test1"):
    image_data=cv2.imread(os.path.join("../input/test1/test1",file), cv2.IMREAD_GRAYSCALE)
    try:
        image_data=cv2.resize(image_data,(96,96))
        test_data.append(image_data/255)
        id.append((file.split("."))[0])
    except:
        print ("______")
    counter+=1
    if counter%1000==0:
        print (counter," image data retreived")

In [ ]:
test_data1=np.array(test_data)
print (test_data1.shape)
test_data1=test_data1.reshape((test_data1.shape)[0],(test_data1.shape)[1],(test_data1.shape)[2],1)

In [ ]:
dataframe_output=pd.DataFrame({"id":id})

In [ ]:
predicted_labels=model.predict(test_data1)

In [ ]:
predicted_labels=np.round(predicted_labels,decimals=2)

In [ ]:
labels=[1 if value>0.5 else 0 for value in predicted_labels]

In [ ]:
dataframe_output["label"]=labels

In [ ]:
dataframe_output.to_csv("submission.csv",index=False)